In [1]:
import os
import requests

pdf_path = r"data\human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path
  response = requests.get(url)

  if response.status_code == 200:
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File doesn't exist, downloading...
The file has been downloaded and saved as data\human-nutrition-text.pdf


In [2]:
import fitz 
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number + 1,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

/home/ubaid/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
1208it [00:01, 979.00it/s]


[{'page_number': 1,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': 2,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [7]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 964,
  'page_char_count': 1596,
  'page_word_count': 281,
  'page_sentence_count_raw': 13,
  'page_token_count': 399.0,
  'text': 'should be lean, and healthy fats, such as omega-3 fatty acids, are  part of any good diet.  Micronutrients  An increase in certain micronutrients can help maintain health  during this life stage. The recommendations for calcium increase to  1,200 milligrams per day for both men and women to slow bone loss.  Also to help protect bones, vitamin D recommendations increase  to 10–15 micrograms per day for men and women. Vitamin B6  recommendations rise to 1.7 milligrams per day for older men and  1.5 milligrams per day for older women to help lower levels of  homocysteine and protect against cardiovascular disease. As adults  age, the production of stomach acid can decrease and lead to an  overgrowth of bacteria in the small intestine. This can affect the  absorption of vitamin B12\xa0and cause a deficiency. As a result, older  adults need more